<a href="https://colab.research.google.com/github/cstecker/politicsRLab/blob/main/sozial%20verzerrte%20Wahlbeteiligung%20(OB-Wahl%20FFM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sozial ungleiche Wahlbeteiligung**

**Alle Befehle bzw. Code-Schnipsel können Sie mit *Strg+Enter* oder durch das klicken des Play-Buttons passagenweise ausführen.** Darunter sehen Sie dann jeweils welches Ergebnis die jeweiligen Zeilen produzieren.
Sie können die Zeilen auch verändern und schauen, wie sich der Code verändert.

Laden wir uns zunächst die Library mit den Befehlen, die wir bei der Analyse benötigen. Libraries sind Bibliotheken von Befehlen. Das tidyverse ist eine ganze Sammlung von Paketen, die insbesondere von Hadley Wickham, einem Superstar in der R-Welt, geprägt wurden:

In [ ]:
library(tidyverse)

Nun laden wir die Datensätze. (Die aufbereiteten Datensätze werden auf github bereitgestellt. Die Details hierzu müssen uns erst einmal nicht weiter kümmern).

In [8]:
# Datensätze herunterladen -------------------------------------------------------------
download.file("https://github.com/cstecker/politicsRLab/blob/main/data/ffm_ob_wbz_results.Rda?raw=true", "ffm_ob_wbz_results.Rda")
download.file("https://github.com/cstecker/politicsRLab/blob/main/data/ffm_stadtteil_2023.Rda?raw=true", "ffm_stadtteil_2023.Rda")
download.file("https://github.com/cstecker/politicsRLab/blob/main/data/ffm_wbz_stadtteile.Rda?raw=true", "ffm_wbz_stadtteile.Rda")

In [ ]:
list.files()

In [7]:
# Datensätze laden
load("ffm_ob_wbz_results.Rda")
load("ffm_stadtteil_2023.Rda")
load("ffm_wbz_stadtteile.Rda")

In [ ]:
# Betrachten der Daten
ffm_ob_wbz_results

In [ ]:
# Betrachten der Daten
wbz_stadtteile

In [ ]:
# Betrachten der Daten
ffm_socstrk

## soziale Ungleichheit in Frankfurt

In [ ]:
# Wohnflaeche
ffm_socstrk %>%
  ggplot(aes(x = wohnflaech, y = stadtteil)) +
  geom_col() 

In [ ]:
# Wählen Sie einen eigenen Indikator
ffm_socstrk %>%
  ggplot(aes(x = ..., y = stadtteil)) +
  geom_col() 

In [ ]:
# alles auf einen Blick
ffm_socstrk %>%
  pivot_longer(-stadtteil) 

# Sozial ungleiches Wahlverhalten

In [ ]:
ffm_ob_wbz_results %>% 
  filter(date == "2023-03-05", wbz_art == "Präsenzwahl") %>% 
  left_join(wbz_stadtteile) %>% 
  select(date, stadtteil, wbz_nr, everything()) %>% 
  group_by(stadtteil) %>% 
  summarise(across(c(gueltig, sperrW , wahlberechtigt), sum)) %>% 
  mutate(turnout = (gueltig+sperrW)/wahlberechtigt*100) %>% 
  left_join(select(ffm_socstrk, -mean_age)) %>% 
  drop_na(stadtteil) %>%
  View()

In [ ]:
ffm_ob_wbz_results %>% 
  filter(date == "2023-03-05", wbz_art == "Präsenzwahl") %>% 
  left_join(wbz_stadtteile) %>% 
  select(date, stadtteil, wbz_nr, everything()) %>% 
  group_by(stadtteil) %>% 
  summarise(across(c(gueltig, sperrW , wahlberechtigt), sum)) %>% 
  mutate(turnout = (gueltig+sperrW)/wahlberechtigt*100) %>% 
  left_join(select(ffm_socstrk, -mean_age)) %>% 
  drop_na(stadtteil) %>%
  ggplot(aes(x = wohnflaech, y = turnout)) +
  geom_point()

In [ ]:
# alles auf einen Blick
ffm_ob_wbz_results %>% 
  filter(date == "2023-03-05", wbz_art == "Präsenzwahl") %>% 
  left_join(wbz_stadtteile) %>% 
  select(date, stadtteil, wbz_nr, everything()) %>% 
  group_by(stadtteil) %>% 
  summarise(across(c(gueltig, sperrW , wahlberechtigt), sum)) %>% 
  mutate(turnout = (gueltig+sperrW)/wahlberechtigt*100) %>% 
  left_join(select(ffm_socstrk, -mean_age)) %>% 
  drop_na(stadtteil) %>%
  pivot_longer(c(auslaender:median_brutto)) %>% 
  mutate(value = replace(value, value == 0, NA)) %>% 
  mutate(name = factor(name, levels = c("median_brutto", "auslaender", "migrant", "wohnflaech", "gymn", "arbeitslose"))) %>% 
  ggplot(aes(x = value, y =  turnout)) +
  geom_ribbon(stat='smooth', method = "lm", se=TRUE, alpha=0.1, fill = "grey60",
              aes(color = NULL)) +
  geom_line(stat="smooth", method = "lm",
            size = .5,
            linetype ="dashed",
            alpha = 0.5,
            color = "grey25") +
  geom_point(color = "#D95F02", alpha = .7) +
  facet_wrap(vars(name), scales = "free",
             labeller = 
               labeller(name = 
                          c("median_brutto" = "Medianeinkommen",
                            "arbeitslose" = "Arbeitslosendichte" , 
                            "auslaender" = "Ausländeranteil", 
                            "gymn" = "Übergänge zum Gymnasium", 
                            "migrant" = "Anteil Migrationshintergrund",
                            "wohnflaech" = "Wohnfläche pro Kopf"))) +
  labs(x = "", y = "Wahlbeteiligung (%)") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(size = 8)
  )
  

# Regressionsanalyse

In [ ]:
# Vorbereiten der Daten
reg_data <- ffm_ob_wbz_results %>% 
  filter(date == "2023-03-05", wbz_art == "Präsenzwahl") %>% 
  left_join(wbz_stadtteile) %>% 
  select(date, stadtteil, wbz_nr, everything()) %>% 
  group_by(stadtteil) %>% 
  summarise(across(c(gueltig, sperrW , wahlberechtigt), sum)) %>% 
  mutate(turnout = (gueltig+sperrW)/wahlberechtigt*100) %>% 
  left_join(select(ffm_socstrk, -mean_age)) %>% 
  drop_na(stadtteil) 
reg_data

In [43]:
# Regression
model1 <- lm(turnout ~ median_brutto, data = reg_data)
model1


Call:
lm(formula = turnout ~ median_brutto, data = reg_data)

Coefficients:
  (Intercept)  median_brutto  
     5.444103       0.009054  


In [42]:
library(broom)

In [45]:
tidy(model1)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),5.444102640,5.460082132,0.9970734,3.244389e-01
median_brutto,0.009053752,0.001323456,6.8409903,2.454717e-08


In [ ]:
tidy(model1) %>%
  filter(term != "(Intercept)") %>%
  ggplot(aes(x = estimate, y = term)) +
  geom_point()